In [2]:
# import dfTrain and dfTest
import pandas as pd
dfTrain = pd.read_csv("dfTrain.csv")
dfTest = pd.read_csv("dfTest.csv")

In [3]:
# remove unnecessary lines in the email, so we remove lines that starts with smth. We create a csv file that does that
dfRemoveStartsWithRow = pd.read_csv("startsWith.csv")
#list_dfRemoveStart = dfRemoveStartsWithRow['Starting'].tolist()
#list_dfRemoveStart

['Message-ID:',
 'Date:',
 'From:',
 'To:',
 'Subject:',
 'Mime-Version:',
 'Content-Type:',
 'Content-Transfer-Encoding:',
 'X-From:',
 'X-To:',
 'X-cc:',
 'X-bcc:',
 'X-Folder:',
 'X-Origin:',
 'X-FileName:']

In [5]:
# https://stackoverflow.com/questions/22245171/how-to-lowercase-a-pandas-dataframe-string-column-if-it-has-missing-values
# change everything in the df to lower case if they are str
dfTrain = dfTrain.applymap(lambda s:s.lower() if type(s) == str else s)
dfTest = dfTest.applymap(lambda s:s.lower() if type(s) == str else s)
dfRemoveStartsWithRow = dfRemoveStartsWithRow.applymap(lambda s:s.lower() if type(s) == str else s)

# Use NLTK for preprocessing

In [28]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
from itertools import chain
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

def categories_and_length(df):
    uniqueCategories = df.Area.unique()
    print(len(uniqueCategories))
    
    for category in uniqueCategories:
        dfCat = df[df.Area== category]
        print('category: {} , len {}'.format(category, len(dfCat)))
        
def processDfRemoveStopWords(df, dataType):
    '''
    df use dfTrain and dfTest dataset
    dataType use "TEST" or "TRAIN" used when saving file name
    '''
    df_list_SR = df['SR Details'].tolist()
    
    tupAppend = []
    for rows in range(0, len(df)):
        perrow = df_list_SR[rows]
        splitString = str(perrow).split("\r\n")
        
        wordsWOStopWords = []
        for word in splitString:
            tupAppend.append((rows, word))
    print(tupAppend[0:10])
    
    # convert the tuples into a df to easily access
    dfTuple = pd.DataFrame(tupAppend, columns = ['CellIdx','Sentence'])
    
    # drop empty "Sentence"
    dfTuple = dfTuple[dfTuple['Sentence']!=""]
    
    # iterate over each row
    listOfIdxToDrop = []
    # we want the index as well
    for row in dfTuple.iterrows():
        sentence = row[1].Sentence
        Idx = row[0]
        #print("sentence : {}".format(sentence))
        
        for token in dfRemoveStartsWithRow.Starting:
            if sentence.startswith(token):
                listOfIdxToDrop.append(Idx)
    # time to drop the relevant rows that starts with the words in dfRemoveStartsWithRow
    print('Len before dropping: {}'.format(len(dfTuple)))
    dfTupleDrop = dfTuple.drop(listOfIdxToDrop)
    print('Len after dropping : {}'.format(len(dfTupleDrop)))
    
    # keeping cell idx, merge all rows with same cellIdx
    # some of the sentences may have been removed and hence whole email gone
    # combine rows with same CellIdx into one long sentence
    numUniIdx = dfTupleDrop.CellIdx.unique()
    
    # first sort the CellIdx in running order
    dfTupleSort = dfTupleDrop.sort_values(by = "CellIdx")
    dfTupleSort2Col = dfTupleSort.reset_index(drop = True)
    
    # Merge sentences if they belong to same index
    dfOut = dfTupleSort2Col.groupby('CellIdx')['Sentence'].apply(lambda x:"{%s}" % ', '.join(x))
    
    # convert series to dataframe
    dfout = dfOut.to_frame()
    fnToSave = 'output/mergedSentencesBeforeToken_{}.csv'.format(dataType)
    dfout.to_csv(fnToSave)
    
    # create index columnname for the df dataframe
    df['new_id'] = df.index
    
    # to make the sentences into list of sentence and now we only need cell ordering
    # read the CSV output from above, which is tokenised to sentences by CellIdx
    dfout = pd.read_csv(fnToSave)
    
    # final DF that we will work on NLTK
    df_3Col = pd.merge(dfout, df, how ='inner', left_on ='CellIdx', right_on = 'new_id')
    
    ## send df_3Col to a csvfile
    #finalEmailFn = 'output/finalEmailArea_{}.csv'.format(dataType)
    #df_3Col.to_csv(finalEmailFn)
    
    # tokenise into sentence
    # at this point it is ok to mess up the words and sentences orders    
    dfEmailArea = df_3Col
    
    # tokenise into words
    sents1 = []
    for text in dfEmailArea['Sentence']:
        sents1.append(sent_tokenize(text))
    sentsUnlist1 = [' '.join(item) for item in sents1]
    sents1 = sentsUnlist1
    words1 = [word_tokenize(sent) for sent in sents1]
    dfEmailArea = dfEmailArea.assign(tokenised = words1)
    
    # remove stopwords
    customStopWords = set(stopwords.words('english')+list(punctuation))
    wordsWOStopwords = []
    for text in dfEmailArea['tokenised']:
        text = " ".join(text)
        wordsWOStopwords.append([word for word in word_tokenize(text.lower()) if word not in customStopWords]) ##
        
    dfEmailArea = dfEmailArea.assign(removeStopwords = wordsWOStopwords)
    wordsWOStopwords = list(chain(*wordsWOStopwords)) # unlist a list of lists
    
    # Stemming words / Lemmatizing
    # stemming the column "removeStopwords" which is the final column after removing Stopwords
    st = LancasterStemmer()
    # as opposed to stemming, we use LEMMATIZER
    
    lemmatizer = WordNetLemmatizer()
    
    LemWord = []
    for sentence in dfEmailArea['removeStopwords']:
        sentence = " ".join(sentence)
        lemWords = [lemmatizer.lemmatize(word) for word in word_tokenize(sentence)]
        LemWord.append([stem for stem in lemWords])
    dfEmailArea = dfEmailArea.assign(lemmatizedWords = LemWord)
    
    # merge each row of "Sentence" back in 1 sentence instead of list
    lenDF = len(dfEmailArea)
    listMergeAll = []
    for i in range(0,lenDF):
        lemWorded = dfEmailArea.lemmatizedWords.iloc[i]
        mergestr = " ".join(lemWorded)
        listMergeAll.append(mergestr)
        
    dfEmailArea = dfEmailArea.assign(mergeAll = listMergeAll)
    
    return dfEmailArea
    

In [29]:
# Preprocessing for dfTrain and dfTest
dftrain = processDfRemoveStopWords(dfTrain, 'TRAIN')
dftest = processDfRemoveStopWords(dfTest, 'TEST')

[(0, 'message-id: <15464986.1075855378456.javamail.evans@thyme>'), (0, 'date: fri, 4 may 2001 13:51:00 -0700 (pdt)'), (0, 'from: phillip.allen@enron.com'), (0, 'to: john.lavorato@enron.com'), (0, 'subject: re:'), (0, 'mime-version: 1.0'), (0, 'content-type: text/plain; charset=us-ascii'), (0, 'content-transfer-encoding: 7bit'), (0, 'x-from: phillip k allen'), (0, 'x-to: john j lavorato <john j lavorato/enron@enronxgate@enron>')]
Len before dropping: 35261
Len after dropping : 19354
[(0, 'message-id: <3555083.1075855675348.javamail.evans@thyme>'), (0, 'date: wed, 9 aug 2000 06:28:00 -0700 (pdt)'), (0, 'from: phillip.allen@enron.com'), (0, 'to: stagecoachmama@hotmail.com'), (0, 'subject: '), (0, 'mime-version: 1.0'), (0, 'content-type: text/plain; charset=us-ascii'), (0, 'content-transfer-encoding: 7bit'), (0, 'x-from: phillip k allen'), (0, 'x-to: stagecoachmama@hotmail.com')]
Len before dropping: 9741
Len after dropping : 6433


# classification using Decision Tree

In [66]:
dftest
# export dftrain later i need to import for plotting decision tree
dftrain.to_csv("output/dftrain.csv")

In [45]:
X = dftrain['mergeAll']
Y = dftrain['Area']
print(x.shape)
print(y.shape)

(1000,)
(1000,)


In [38]:
# remove sparse terms
X1 = remove_zero_tf_idf(X)

In [67]:
# X1 is the new X
# Y is stil Y

# now we fit the model decision tree
from datetime import datetime
import pickle
from sklearn.pipeline import Pipeline
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# modelling
text_clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,2))),
    ('tfidf',TfidfTransformer()),
    ('chi2sampler',AdditiveChi2Sampler(sample_steps=2)),
    ('clf',tree.DecisionTreeClassifier()),
    
])

parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'clf__alpha':(1e-4),
}

In [68]:
# fitting the pipeline
text_clf.fit(X,Y)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [69]:
# save the model
savename_text_clf = 'output/fittedmodel_decisionTree.pickle'
with open(savename_text_clf,'wb') as fp:
    pickle.dump(text_clf,fp)

In [56]:
# import test(train test) set for prediction
predicted = text_clf.predict(dftest['mergeAll'])
dftest = dftest.assign(predicted = predicted)

In [59]:
# dftest is the testing on part of the train set in reality we do that, usually ppl random select 30% from train set
# as the testing set. In my case, i just take a separate set frm the whole set since i didnt want to use the super big dataset
# for this demonstration on github

# report the AUC metrics with F1 scores
from roc_auc_f1_score_metrics import class_report
y_score = text_clf.predict_proba(dftest['mergeAll'])

report_with_auc = class_report(
    y_true = dftest.Area,
    y_pred = text_clf.predict(dftest['mergeAll']),
    y_score = y_score
)
print(report_with_auc)


             precision    recall  f1-score  support   pred       AUC
cas           0.260417  0.431034  0.324675     58.0   96.0  0.478056
cash          0.309091  0.250000  0.276423     68.0   55.0  0.478936
trade         0.480000  0.320000  0.384000     75.0   50.0  0.561905
avg / total   0.358818  0.328358  0.330487    201.0  201.0  0.500483


In [65]:
# print the metric to a csv file for keep
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.multiclass import unique_labels
report_name = 'output/f1_Score_ROC_decisionTree.csv'

# constructing the dataframe
labels = unique_labels(dftest.Area, predicted)
precision,recall,fscore, support = metrics.precision_recall_fscore_support(dftest.Area, predicted, labels= labels)

result_pd = pd.DataFrame({
    "class": labels,
    "precision": precision,
    "recall": recall,
    "f1-score":fscore,
    "support":support
})

# save file
result_pd.to_csv(report_name)

# here and below are test codes